<a href="https://colab.research.google.com/github/NeuralClassifier/ReusabilityRate_ML/blob/main/CMPUT_663_ReuseRate_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import  LogisticRegression
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import SGDRegressor
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold 